In [1]:
import re
import nltk
import sys
import csv
import heapq

In [ ]:
from flask import Flask, request, render_template
app = Flask(__name__)


@app.route("/")
def home():
    return render_template('summary.html')

@app.route('/get_text', methods=['POST'])

def get_text():
    output = request.form.to_dict()
    text = output["text1"]
    article_text = re.sub(r'\[[0-9]*\]', ' ', text)
    article_text = re.sub(r'\s+', ' ', article_text)
    format_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    format_article_text = re.sub(r'\s+', ' ', format_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    word_freq = {}
    for word in nltk.word_tokenize(format_article_text):
        if word not in stopwords:
            if word not in word_freq.keys():
                word_freq[word] = 1
            else:
                word_freq[word] +=1
    max_freq = max(word_freq.values())
    for word in word_freq.keys():
        word_freq[word] = (word_freq[word]/max_freq)
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_freq.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_freq[word]
                    else:
                        sentence_scores[sent] += word_freq[word]
    summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)                    
    
    return render_template('summary.html',summ=summary,inputt=artical_text)
    
    
if __name__ == '__main__':
    app.run(debug = True,use_reloader=False)